In [1]:
import os
import glob
import numpy as np
import pandas as pd
import read_exported

# Get Ephys timestamp data

In [55]:
# get all the .rec filesi n the data folder adn add their info to a dictionary
trodes_res = read_exported.organize_all_trodes_export(r"F:\data")

for i in trodes_res.keys():
    # get the data from the dictionary
    voltage_timestamp_arr = trodes_res[i]['raw']['timestamps']['data']
    dio_arr = trodes_res[i]["DIO"]

    nosepoke_lst = []
    key_lst = []
    for key, value in dio_arr.items():
        # if "in" in key: # only get the nosepoke data
        val_arr = value['data'] # get the array
        nosepoke_lst.append(val_arr) # append the array
        key_lst = key_lst + ([key] * val_arr.shape[0]) # repeat the key for each row in the array
    nosepoke_arr = np.concatenate(nosepoke_lst, axis=0) # concatenate all the arrays
    df = pd.DataFrame(nosepoke_arr) # convert to dataframe
    df['key'] = key_lst # add the key column
    df_wide = df.pivot(index='time', columns='key', values='state').reset_index() # pivot the dataframe
    df_wide.to_feather(i + '.feather')
    # add all the timestamps to the dataframe
    timestamp_df = pd.DataFrame(voltage_timestamp_arr) # convert to dataframe
    timestamp_df = pd.DataFrame([timestamp_df.min(), timestamp_df.max()]) # only keep the highest number and lowest number to set the range
    all_time_df = pd.merge(timestamp_df, df_wide, on='time', how='outer') # merge the dataframes
    all_time_df = all_time_df.sort_values(by='time').reset_index(drop=True)

d:\GIT_REPOS\cmarais_neuro\03_Scripts\read_exported.py:62: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dtype_spec)


Skipping file 20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.timestampoffset.txt due to error: Settings format not supported
Skipping file 20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged.timestampoffset.txt due to error: Settings format not supported


# Get Video timestamp data and frames of each video

In [48]:
vid_files = glob.glob(os.path.join(r"F:\data", "*cameraHWSync"))
df_lst = []
for i in range(len(vid_files)):
    vid_df = pd.DataFrame(read_exported.read_trodes_extracted_data_file(vid_files[i])["data"]["PosTimestamp"]).drop_duplicates().reset_index(drop=True)
    vid_df = vid_df.reset_index().rename(columns={'index': 'frame', 0: 'time'})
    vid_df['video_num'] = "video_num_" + str(i+1)
    df_lst.append(vid_df)
all_vid_df = pd.concat(df_lst, axis=0).reset_index(drop=True)
all_vid_wide_df = all_vid_df.pivot(index='time', columns='video_num', values='frame')
all_vid_wide_df



d:\GIT_REPOS\cmarais_neuro\03_Scripts\read_exported.py:62: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dtype_spec)
d:\GIT_REPOS\cmarais_neuro\03_Scripts\read_exported.py:62: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dtype_spec)


video_num,video_num_1,video_num_2
time,,
835678,0.0,0.0
837064,1.0,1.0
838450,2.0,2.0
838921,NaN,3.0
839835,3.0,4.0
...,...,...
69425134,52443.0,52793.0
69426520,52444.0,52794.0
69427133,NaN,52795.0


# Merge Ephys data with video data

In [59]:
all_df = pd.merge(all_time_df, all_vid_wide_df, on='time', how='outer').sort_values(by='time').reset_index(drop=True)
all_df

,time,dio_ECU_Din1,dio_ECU_Din2,dio_ECU_Din3,dio_ECU_Din4,dio_ECU_Dout1,dio_ECU_Dout2,dio_ECU_Dout3,dio_ECU_Dout4,video_num_1,video_num_2
0,835678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,835680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,837064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
3,838450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0
4,838921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...
59662,69425134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52443.0,52793.0
59663,69426520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52444.0,52794.0
59664,69427133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52795.0
59665,69427906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52445.0,52796.0


In [88]:
all_df['frame'] = ((all_df['time'] - all_df['time'].min())/1300).round(0).astype(int)
all_df

,time,dio_ECU_Din1,dio_ECU_Din2,dio_ECU_Din3,dio_ECU_Din4,dio_ECU_Dout1,dio_ECU_Dout2,dio_ECU_Dout3,dio_ECU_Dout4,video_num_1,video_num_2,frame
0,835678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0
1,835680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0
2,837064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1
3,838450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2
4,838921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
59662,69425134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52443.0,52793.0,52761
59663,69426520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52444.0,52794.0,52762
59664,69427133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52795.0,52763
59665,69427906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52445.0,52796.0,52763


In [80]:
pd.DataFrame(all_vid_wide_df['video_num_2'].dropna().index).diff().mean()

time    1299.195295
dtype: float64

In [ ]:
###### double check to make sure the timestamps are correct and the same as original df in LEOs code
###### timestamp index is the timestamp - smallest timestamp
########### Add camera timestamps and nosepokes to the matrix
########### Save amtrix in feather format
########### allign box outputs video ouitputs
########### rewrite extraction code to bne more direct to getting data from trodes